In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
from dateutil import tz
from pathlib import Path
import yaml
from nelson_lab_to_nwb.blackrock_sessions_converter import BlackrockNWBConverter
from neuroconv.datainterfaces import BlackrockRecordingInterface, BlackrockSortingInterface
from neuroconv.utils.dict import load_dict_from_file, dict_deep_update

In [3]:
# set the source data for the Converter
source_data = dict(
    BlackrockRaw=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6"
    ),
    BlackrockLFP=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns2"
    ),
    BlackrockSorting=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.nev",
        sampling_frequency=30000,
    )
)

# Instantiate the Converter
converter = BlackrockNWBConverter(source_data)

# Automatically fetch metadata from files, then update it with user-defined metadata
source_metadata = converter.get_metadata()
user_metadata_file = "/mnt/shared_storage/Github/nelson-lab-to-nwb/src/nelson_lab_to_nwb/blackrock_sessions_converter/metadata_example.yaml"
user_metadata = load_dict_from_file(file_path=user_metadata_file)
metadata = dict_deep_update(source_metadata, user_metadata)

Source data is valid!


In [4]:
print(metadata)

DeepDict: {'NWBFile': {'session_description': 'Blackrock recording session', 'identifier': '3b7182f0-21fa-413e-b6f4-7cec100ff422', 'session_start_time': '2023-07-18T19:33:06', 'institution': 'University of California San Francisco', 'lab': 'Nelson Lab', 'experimenter': ['Last, First Middle']}, 'Ecephys': {'Device': [{'name': 'DeviceEcephys', 'description': 'no description'}], 'ElectrodeGroup': [{'name': 'ElectrodeGroup', 'description': 'no description', 'location': 'unknown', 'device': 'DeviceEcephys'}], 'ElectricalSeries': {'name': 'ElectricalSeries', 'description': 'Acquisition traces for the ElectricalSeries.'}}, 'Subject': {'subject_id': 'subid012345', 'genotype': 'not defined', 'description': 'not defined', 'species': 'Mus musculus', 'date_of_birth': '2023-03-17T00:00:00', 'sex': 'F'}}


In [5]:
# Conversion options
conversion_options = dict(
    BlackrockRaw=dict(
        stub_test=True,
    ),
    BlackrockLFP=dict(
        write_as="lfp"
    )
)

# Run conversion
converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_blackrock.nwb",
    overwrite=True,
    conversion_options=conversion_options
)

Metadata is valid!
conversion_options is valid!


/home/luiz/anaconda3/envs/nelson_lab_to_nwb_env/lib/python3.12/site-packages/pynwb/file.py:146: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['date_of_birth'] = _add_missing_timezone(date_of_birth)
/home/luiz/anaconda3/envs/nelson_lab_to_nwb_env/lib/python3.12/site-packages/pynwb/file.py:471: UserWarning: Date is missing timezone information. Updating to local timezone.
  args_to_set['session_start_time'] = _add_missing_timezone(session_start_time)


NWB file saved at converted_blackrock.nwb!


In [6]:
from pynwb import NWBHDF5IO
from nwbwidgets import nwb2widget

io = NWBHDF5IO("converted_blackrock.nwb", mode='r')
nwb = io.read()

nwb2widget(nwb)

In [7]:
nwb

,location,group,group_name,channel_name,gain_to_uV,offset_to_uV
id,,,,,,
0,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig001,0.25,0.0
1,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig002,0.25,0.0
2,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig003,0.25,0.0
3,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig004,0.25,0.0
,location,group,group_name,channel_name,gain_to_uV,offset_to_uV
id,,,,,,
0,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig001,0.25,0.0
1,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig002,0.25,0.0
2,unknown,ElectrodeGroup pynwb.ecephys.ElectrodeGroup at 0x140157625390992\nFields:\n description: no description\n device: DeviceEcephys pynwb.device.Device at 0x140157627037792\nFields:\n description: no description\n\n location: unknown\n,ElectrodeGroup,sig003,0.25,0.0


# Data Interfaces

In [ ]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6"

interface = BlackrockRecordingInterface(file_path=file_path, verbose=False)

# Extract what metadata we can from the source files
metadata = interface.get_metadata()

# # For data provenance we add the time zone information to the conversion
# session_start_time = metadata["NWBFile"]["session_start_time"]
# session_start_time = session_start_time.replace(tzinfo=tz.gettz("US/Pacific")).isoformat()
# metadata["NWBFile"].update(session_start_time=session_start_time)

# # Choose a path for saving the nwb file and run the conversion
# nwbfile_path = f"{path_to_save_nwbfile}"
# interface.run_conversion(nwbfile_path=nwbfile_path, metadata=metadata)

In [ ]:
rec = interface.recording_extractor
nr = rec.neo_reader

In [ ]:
rec

In [ ]:
interface.get_metadata()

In [ ]:
file_path = "/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.nev"

sorting_interface = BlackrockSortingInterface(
    file_path=file_path,
    sampling_frequency=30000,
    verbose=False
)

In [ ]:
se = sorting_interface.sorting_extractor

In [ ]:
se.to_spike_vector()

In [ ]:
from neuroconv.datainterfaces.ecephys.blackrock.header_tools import parse_nsx_basic_header

In [ ]:
h = parse_nsx_basic_header("/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6")

In [ ]:
h

# Random tests

In [ ]:
from neuroconv.datainterfaces import BlackrockRecordingInterface
from neuroconv import NWBConverter

class BlackrockNWBConverter(NWBConverter):
    data_interface_classes = dict(BlackrockRaw=BlackrockRecordingInterface)

In [ ]:
source_data = dict(
    BlackrockRaw=dict(
        file_path="/media/luiz/Seagate Basic/storage/catalystneuro/nelson_lab/Raw_BlackRock_withCognitiveEvents/BlackRock_raw_H0.2_071823001.ns6"
    )
)

converter = BlackrockNWBConverter(source_data)

source_data = dict(BlackrockRaw=dict(stub_test=True))

converter.run_conversion(
    metadata=metadata,
    nwbfile_path="converted_blackrock.nwb",
    overwrite=True
)